# 0. Modules



In [ ]:
# 1. Data Import
from google.colab import files
import pandas as pd
# 2. Text Preprocessing
import collections
import nltk
import re
from nltk import ngrams
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedShuffleSplit
import collections as co
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from nltk.tokenize import sent_tokenize
from tensorflow.python.keras.preprocessing.text import Tokenizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
# pad sequences
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
# Pre-trained Embeddings
from tqdm import tqdm
import os
# Parameter Tuning
from sklearn.model_selection import GridSearchCV
# 3. Model Architectures
  # Dummy Classifier
from sklearn.dummy import DummyClassifier
  # for Random Forest
from xgboost import XGBClassifier
  # SVM
from sklearn.svm import LinearSVC
  # Knn
from sklearn.neighbors import KNeighborsClassifier
  # for all neural networks 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, Flatten, BatchNormalization
  # for cnn
from tensorflow.python.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
  #rnn
from tensorflow.python.keras.layers import LSTM, Input, Concatenate, Bidirectional, concatenate, GlobalMaxPool1D, Flatten
from keras import Model
from keras.layers.merge import concatenate
# To save model weights
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
  # Randomized GridSearch
from sklearn.model_selection import RandomizedSearchCV
# 4. Results
import time
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
import seaborn as sns
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix
from keras.metrics import Precision, Recall
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from tensorflow.keras.utils import plot_model
## for imbalanced dataset
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

import csv

# For plotly figures
from plotly.offline import iplot, plot, download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go

# 1. Data Import and Cleaning.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


> First choose scenario.

Compare results with CoNLL-2010 Shared Task's Task1W or with only the Dummy Classifier ? 

We must add extra non-weasel sentences to our data, so that the ratio is comparable.

In [ ]:
print('Compare results with Task1W results from the CoNLL-2010 Shared Task ?')
print('Answer yes or no')
scenario = input()
if scenario == 'yes':
    print('Extra non-weasel sentences will be added.') 
elif scenario == 'no':
    print('Data will remain the same.') 
else:
    print('Incorrect Input.')   

Compare results with Task1W results from the CoNLL-2010 Shared Task ?
Answer yes or no
no
Data will remain the same.


In [ ]:
raw_df = pd.read_csv('drive/MyDrive/clean_dataset.csv')

In [ ]:
print(raw_df.shape)

(91914, 4)


In [ ]:
# In case for 1st scenario.
if scenario == 'yes':
    extra_df = pd.read_csv('drive/MyDrive/extra_dataset.csv')
    print(extra_df.shape)



> Clean text



In [ ]:
# Convert sentences to lowercase
raw_df['Weasel_Sentence'] = raw_df['Weasel_Sentence'].str.lower()
raw_df['Updated_Sentence'] = raw_df['Updated_Sentence'].str.lower()

# In case for 1st scenario.
if scenario == 'yes':
    extra_df['Non_Weasel_Sentence'] = extra_df['Non_Weasel_Sentence'].str.lower()

In [ ]:
# Remove numbers and commas, dots.
raw_df['Weasel_Sentence'] = raw_df['Weasel_Sentence'].str.replace('\d+', '')
raw_df['Updated_Sentence'] = raw_df['Updated_Sentence'].str.replace('\d+', '')

raw_df['Weasel_Sentence'] = raw_df['Weasel_Sentence'].str.replace('.', '')
raw_df['Updated_Sentence'] = raw_df['Updated_Sentence'].str.replace('.', '')

raw_df['Weasel_Sentence'] = raw_df['Weasel_Sentence'].str.replace(',', '')
raw_df['Updated_Sentence'] = raw_df['Updated_Sentence'].str.replace(',', '')

raw_df['Weasel_Sentence'] = raw_df['Weasel_Sentence'].str.replace('%', '')
raw_df['Updated_Sentence'] = raw_df['Updated_Sentence'].str.replace('%', '')

# In case for 1st scenario.
if scenario == 'yes':
    extra_df['Non_Weasel_Sentence'] = extra_df['Non_Weasel_Sentence'].str.replace('\d+', '')
    extra_df['Non_Weasel_Sentence'] = extra_df['Non_Weasel_Sentence'].str.replace('.', '')
    extra_df['Non_Weasel_Sentence'] = extra_df['Non_Weasel_Sentence'].str.replace('%', '')

In [ ]:
# Remove consecutive [reference] tags. One is enough.

raw_df['Weasel_Sentence'] = raw_df['Weasel_Sentence'].apply(lambda x: re.sub(r'(\s*\[(reference)\]\s*)\1{1,}', '\\1', str(x)))
raw_df['Updated_Sentence'] = raw_df['Updated_Sentence'].apply(lambda x: re.sub(r'(\s*\[(reference)\]\s*)\1{1,}', '\\1', str(x)))

# In case for 1st scenario.
if scenario == 'yes':
    extra_df['Non_Weasel_Sentence'] = extra_df['Non_Weasel_Sentence'].apply(lambda x: re.sub(r'(\s*\[(reference)\]\s*)\1{1,}', '\\1', str(x)))

Remove weasel sentences with less than 3 words, as well as their equivalent non-weasel sentences.

In [ ]:
raw_df = raw_df[raw_df['Weasel_Sentence'].str.split().str.len().ge(3)]
print(raw_df.shape[0])

91627


Do the same for non-weasel sentences.

In [ ]:
raw_df = raw_df[raw_df['Updated_Sentence'].str.split().str.len().ge(3)]
print(raw_df.shape[0])

91464


In [ ]:
# In case for 1st scenario.
if scenario == 'yes':
    extra_df = extra_df[extra_df['Non_Weasel_Sentence'].str.split().str.len().ge(3)]
    print(extra_df.shape[0])

Remove weasel sentences with length more than 99% of all sentence lenghts, as well as their equivalent non-weasel sentences.

In [ ]:
# Find all sentence lengths.
all_sent_lengths =[len(str(sentence).split()) for sentence in raw_df['Weasel_Sentence']]
all_sent_lengths.extend([len(str(sentence).split()) for sentence in raw_df['Updated_Sentence']])
# Find 99% quartile.
q = pd.Series(all_sent_lengths).quantile(0.99)

In [ ]:
raw_df = raw_df[raw_df['Weasel_Sentence'].str.split().str.len().lt(q)]
print(raw_df.shape[0])

90345


Do the same for non-weasel sentences.

In [ ]:
raw_df = raw_df[raw_df['Updated_Sentence'].str.split().str.len().lt(q)]
print(raw_df.shape[0])

90184


In [ ]:
# In case for 1st scenario.
if scenario == 'yes':
    extra_df = extra_df[extra_df['Non_Weasel_Sentence'].str.split().str.len().lt(q)]
    print(extra_df.shape[0])

In [ ]:
# Break dataset to 2 (weasel sentences & non weasel). Then assign a new column that indicates the type of each sentence (weasel or non-weasel). And then concatenate the new dataframes to 1, so that we have the correct form.
#  Note that the concatenation will occur after we split the dataset to train, validation and test. The reason for this is to not separate the weasel sentences with their corrected versions.

def concat(raw_df):
  df_weasel = raw_df[["Title","Weasel_Sentence"]]
  df_weasel.columns = ['title', 'sentence']
  df_weasel.insert(0, 'label', 'weasel')

  df_edited = raw_df[["Title","Updated_Sentence"]]
  df_edited.columns = ['title', 'sentence']
  df_edited.insert(0, 'label', 'not-weasel')

  df = pd.concat([df_weasel,df_edited])
  
  df = df.reset_index()
  X = df['sentence'].astype(str)
  y = df['label'].astype(str)

  return X, y

# 2. Descriptive Statistics

In [ ]:
# Total rows.
print(raw_df.shape[0])

90184


In [ ]:
if not os.path.exists("images"):
    os.mkdir("images")



1.   Histogram of similarity percentages







In [ ]:
fig = go.Figure(data=[go.Histogram(x=raw_df['Similarity_Percentage'],histnorm='percent')])
fig1 = fig.update_xaxes(title_text="Similarity_Percentage")
fig1 = fig.update_yaxes(title_text="Percentage Frequency (%)")
fig1 = fig.update_layout(height=600, width=800, template='plotly_white', title_text="Histogram of Similarity Percentages")
fig1.show()

plot(fig1, filename="images/fig1.html", auto_open=False)

'images/fig1.html'

Summary Statistics on Variable Similarity_Percentage

In [ ]:
raw_df['Similarity_Percentage'].describe()

count    90184.000000
mean         0.660817
std          0.271957
min          0.000000
25%          0.390000
50%          0.640000
75%          0.960000
max          1.000000
Name: Similarity_Percentage, dtype: float64

Percentage of Sentences that are probably exactly the same.

In [ ]:
# Percentage of Sentences that are probably exactly the same.
print((raw_df[raw_df['Similarity_Percentage'] == 1]['Similarity_Percentage'].count()/raw_df.shape[0])*100, "%")

16.02168899139537 %




> For better results, remove weasel and non-weasel sentences that are identical, according to the similarity percentage.



In [ ]:
# Remove rows, where the Similarity percentage is exactly equal to 1.
raw_df = raw_df[raw_df.Similarity_Percentage < 1]
raw_df.shape[0]

75735

3. Sentence Length Histogram

In [ ]:
sent_lengths =[len(str(sentence).split()) for sentence in raw_df['Weasel_Sentence']]
sent_lengths.extend([len(str(sentence).split()) for sentence in raw_df['Updated_Sentence']])

In [ ]:
fig = go.Figure(data=[go.Histogram(x=sent_lengths)])
fig2 = fig.update_xaxes(title_text="Sentence Length")
fig2 = fig.update_yaxes(title_text="Frequency")
fig2 = fig.update_layout(height=600, width=800, template='plotly_white', title_text="Histogram of Sentence Lengths")
fig2.show()

plot(fig2, filename="images/fig2.html", auto_open=False)

'images/fig2.html'


3.   Word Count Descriptive Analysis



First create a corpus for all weasel sentences and a different one for non-weasel sentences.

In [ ]:
# Vocabulary of weasel sentences
corpora_weasel = collections.Counter([y for x in raw_df['Weasel_Sentence'].astype(str).values.flatten() for y in x.split()])
# Vocabulary of not weasel sentences
corpora_not_weasel = collections.Counter([y for x in raw_df['Updated_Sentence'].astype(str).values.flatten() for y in x.split()])

In [ ]:
def get_top_ngram(column, n=1, c=10):
    ng = []
    counts = []
    corpora_weasel = collections.Counter(ngrams([y for x in column.astype(str).values.flatten() for y in x.split()], n))
    ngram_counts = corpora_weasel.most_common(c)
    for i in ngram_counts:
      ngram = " ".join(i[0])
      count = int(i[1])
      ng.append(ngram)
      counts.append(count)

    df = pd.DataFrame(list(zip(ng, counts)), columns =['Grams', 'Frequency'])

    # Barplot.
    trace = go.Bar(
            x=counts,
            y=ng,
            orientation='h')

    return trace

In [ ]:
plot_bigrams = get_top_ngram(raw_df['Weasel_Sentence'], n=2, c=10)
plot_trigrams = get_top_ngram(raw_df['Weasel_Sentence'], n=3, c=10)
plot_4grams = get_top_ngram(raw_df['Weasel_Sentence'], n=4, c=10)
plot_5grams = get_top_ngram(raw_df['Weasel_Sentence'], n=5, c=10)

In [ ]:
# Subplots.
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=[
        'Most common 2-grams', 
        'Most common 3-grams', 
        'Most common 4-grams', 
        'Most common 5-grams'
    ])

fig.add_trace(plot_bigrams, row=1, col=1)
fig.add_trace(plot_trigrams, row=1, col=2)
fig.add_trace(plot_4grams, row=2, col=1)
fig.add_trace(plot_5grams, row=2, col=2)


# Update xaxis properties
fig.update_xaxes(title_text="Number of Occurences", row=1, col=1)
fig.update_xaxes(title_text="Number of Occurences", row=1, col=2)
fig.update_xaxes(title_text="Number of Occurences", row=2, col=1)
fig.update_xaxes(title_text="Number of Occurences", row=2, col=2)

fig3 = fig.update_layout(height=720, width=1280, template='plotly_white', title_text="Barplot of most common n-grams in Weasel-Worded Sentences", 
                  showlegend=False)

fig.show()


plot(fig3, filename="images/fig3.html", auto_open=False)

'images/fig3.html'

# 3. Preprocessing

### Stemming or Lemmatization

In [ ]:
#from nltk.stem import PorterStemmer, WordNetLemmatizer
#
#nltk.download('wordnet')
#import nltk
#
#def stem_sentences(sentence):
#
#    ps = PorterStemmer()
#    lm = WordNetLemmatizer()
#
#    tokens = sentence.split()
#
#    stemmed_tokens = [ps.stem(token) for token in tokens]
#    #lem_tokens = [lm.lemmatize(token) for token in tokens]
#
#    return ' '.join(stemmed_tokens)
##    #return ' '.join(lem_tokens)
#
#raw_df['Weasel_Sentence'] = raw_df['Weasel_Sentence'].apply(lambda x: stem_sentences(x))
#raw_df['Updated_Sentence'] = raw_df['Updated_Sentence'].apply(lambda x: stem_sentences(x))

### Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(raw_df, test_size=0.2)

train, val = train_test_split(train_val, test_size=0.2)

In [ ]:
# In case for 1st scenario, do the same with extra_df.
if scenario == 'yes':
    if 'label' not in extra_df and 'sentence' not in extra_df:
      extra_df.insert(0, 'label', 'not-weasel')
      extra_df.rename(columns={'Non_Weasel_Sentence':'sentence'}, inplace=True)

    extra_train_val, extra_test = train_test_split(extra_df, test_size=0.2)
    extra_train, extra_val = train_test_split(extra_train_val, test_size=0.2)

    extra_X_train, extra_y_train = extra_train.sentence.reset_index(drop=True), extra_train.label.reset_index(drop=True)
    extra_X_val, extra_y_val = extra_val.sentence.reset_index(drop=True), extra_val.label.reset_index(drop=True)
    extra_X_test, extra_y_test = extra_test.sentence.reset_index(drop=True), extra_test.label.reset_index(drop=True)

In [ ]:
print('train_val shape: {}'.format(train_val.shape[0]))
print()

print('train shape: {}'.format(train.shape[0]))
print()

print('val shape: {}'.format(val.shape[0]))
print()

print('test shape: {}'.format(test.shape[0]))

train_val shape: 60588

train shape: 48470

val shape: 12118

test shape: 15147


In [ ]:
X_train, y_train = concat(train)
X_val, y_val = concat(val)
X_test, y_test = concat(test)

> For the 1st scenario

Change ratio of weasels and non-weasels in test data to compare results with the classifiers in the paper https://www.aclweb.org/anthology/W10-3001.pdf

The evaluation dataset contained 2346 Wikipedia paragraphs with 9634 sentences, out of which 2234 were uncertain. So the weasel to non-weasel ratio is approximately 1:4.

So the percentage of weasel sentences in the test dataset must be around 25%, in order to have comparable results.



In [ ]:
# In case for 1st scenario.
if scenario == 'yes':
    X_train = X_train.append(extra_X_train).reset_index(drop=True)
    y_train = y_train.append(extra_y_train).reset_index(drop=True)

    X_val = X_val.append(extra_X_val).reset_index(drop=True)
    y_val = y_val.append(extra_y_val).reset_index(drop=True)

    X_test = X_test.append(extra_X_test).reset_index(drop=True)
    y_test = y_test.append(extra_y_test).reset_index(drop=True)

In [ ]:
print('y_train shape: {}'.format(y_train.shape[0]))
print()

print('y_val shape: {}'.format(y_val.shape[0]))
print()

print('y_test shape: {}'.format(y_test.shape[0]))

y_train shape: 96940

y_val shape: 24236

y_test shape: 30294


In [ ]:
print('train shape: {}'.format(X_train.shape[0]))
print()

print('val shape: {}'.format(X_val.shape[0]))
print()

print('test shape: {}'.format(X_test.shape[0]))

train shape: 96940

val shape: 24236

test shape: 30294


In [ ]:
print('Class percentages on train data:')
print(y_train.value_counts(normalize=True) * 100)
print()

print('Class percentages on val data:')
print(y_val.value_counts(normalize=True) * 100)
print()

print('Class percentages on test data:')
print(y_test.value_counts(normalize=True) * 100)
print()

Class percentages on train data:
not-weasel    50.0
weasel        50.0
Name: label, dtype: float64

Class percentages on val data:
not-weasel    50.0
weasel        50.0
Name: label, dtype: float64

Class percentages on test data:
not-weasel    50.0
weasel        50.0
Name: label, dtype: float64





### Bag of Words Approach (BoW) [Keras Tokenizer + Sentence Padding]



Calculate total vocabulary size of words for all sentences in training data.

In [ ]:
corpora = " ".join(X_train)
total_vocab_size = len(co.Counter(corpora.split()))
print(total_vocab_size)

107793


Choose the number of most used words that the Tokenizer wil keep.

In [ ]:
# In case for 1st scenario.
if scenario == 'yes':
    max_words = 60000
    print('We will only consider the {} most used words in this dataset'.format(max_words))
else:
  max_words = 35000
  print('We will only consider the {} most used words in this dataset'.format(max_words))

We will only consider the 35000 most used words in this dataset


We will now split the unified text into tokens, in order to count them and find the most frequent.
Usually the most frequent tokens in a text are Stop Words

In [ ]:
top_words = co.Counter(corpora.split()).most_common()
top_words[:20]

[('the', 167649),
 ('of', 85670),
 ('and', 63225),
 ('to', 57691),
 ('in', 56420),
 ('a', 49572),
 ('is', 34576),
 ('that', 29514),
 ('as', 27468),
 ('[reference]', 26991),
 ('was', 23553),
 ('for', 19787),
 ('by', 19359),
 ('it', 17556),
 ('with', 15120),
 ('on', 15012),
 ('be', 12411),
 ('some', 12197),
 ('has', 11160),
 ('have', 11051)]

Setting up a Keras Tokenizer & fitting on train data only.

In [ ]:
# Setting up Keras tokenizer
tokenizer = Tokenizer(num_words=max_words, lower=True, oov_token='<OOV>')
# fitting on the train dataset only
tokenizer.fit_on_texts(X_train)

# Tokenizer turns text into sequence of numbers. NOT one-hot-encoding
train_sequences = tokenizer.texts_to_sequences(X_train)
val_sequences = tokenizer.texts_to_sequences(X_val)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
unique_words = tokenizer.word_counts



> Sentence Padding

To proceed, we now have to make sure that all text sequences we feed into the neural network models have the same length.



In [ ]:
# pad_sequences cuts of sequences that are too long and adds zeros to sequences that are too short.

# Check the min length of all sentences
all_train_lengths = list(map(len, train_sequences))

# max length sould be the 95-th percentile of the list of all train sentence lengths.
max_len = int(np.percentile(all_train_lengths, q=95))
print(f'Max Length: {max_len}')


X_train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')

X_val_padded = pad_sequences(val_sequences, maxlen=max_len, padding='post', truncating='post')

X_test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

print('New train data shape: {}'.format(X_train_padded))
print('New validation data shape: {}'.format(X_val_padded))
print('New test data shape: {}'.format(X_test_padded))

Max Length: 48
New train data shape: [[  32  171    9 ...    0    0    0]
 [  26  907    3 ...    0    0    0]
 [  51    2  249 ...    0    0    0]
 ...
 [ 392  352 1324 ...    0    0    0]
 [   6  200    2 ...    0    0    0]
 [  11    2 1677 ...    9  376   54]]
New validation data shape: [[2093  248    1 ...  378    5 4260]
 [   6    1    4 ...    0    0    0]
 [  15    8  114 ...    0    0    0]
 ...
 [  11    1   24 ...    0    0    0]
 [ 142   65 5492 ...    0    0    0]
 [  11    7  752 ...    0    0    0]]
New test data shape: [[   11     1   432 ...     0     0     0]
 [   32    21  5070 ...     0     0     0]
 [   26 13214     8 ...     0     0     0]
 ...
 [   11     6   440 ...     0     0     0]
 [   11    26   213 ...    62     1     1]
 [    1    12   573 ...     0     0     0]]


In [ ]:
print(X_train_padded.shape)
print(X_val_padded.shape)
print(X_test_padded.shape)

(96940, 48)
(24236, 48)
(30294, 48)


### Label Encoder (input for rest classifiers (XGBoost, k-nn, ...))

In [ ]:
from sklearn.preprocessing import LabelEncoder
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_train)


label_encoded_y_train = label_encoder.transform(y_train)
label_encoded_y_test = label_encoder.transform(y_test)
label_encoded_y_val = label_encoder.transform(y_val)

In [ ]:
label_encoded_y_train

array([1, 1, 1, ..., 0, 0, 0])


### One Hot Encoder (input for neural networks)

In [ ]:
# Encoding the labels using a One-Hot Encoder
y_enc = OneHotEncoder(sparse=False)

In [ ]:
# At first we run fit_transform on the Training data
y_train_enc = y_enc.fit_transform(pd.DataFrame(y_train).values.reshape(-1, 1))
y_train_enc

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
# Then we use the fitted One-hot-Encoder to transform the rest of the data
y_val_enc = y_enc.transform(y_val.values.reshape(-1, 1))

y_test_enc = y_enc.transform(y_test.values.reshape(-1, 1))

In [ ]:
print('y_train shape: {}'.format(y_train_enc.shape))
print('y_val shape: {}'.format(y_val_enc.shape))
print('y_test shape: {}'.format(y_test_enc.shape))

y_train shape: (96940, 2)
y_val shape: (24236, 2)
y_test shape: (30294, 2)


In [ ]:
# The categories that we obtained by the Encoder
y_enc.categories_

[array(['not-weasel', 'weasel'], dtype=object)]


## Pre-Trained Word Embeddings (Glove)

In [ ]:
# Download the zip files if they do not exist.
if not os.path.isfile('glove.6B.zip'):
    !wget http://nlp.stanford.edu/data/glove.6B.zip
    !unzip glove*.zip

In [ ]:
#from zipfile import ZipFile

# Create a ZipFile Object and load sample.zip in it
#with ZipFile('glove.6B.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
#   zipObj.extractall()

In [ ]:
word_index = tokenizer.word_index

In [ ]:
def load_glove_embeddings(dim: int = 100) -> dict:
    """
    Function that loads glove embeddings. 

    :param dim: The embeddings size (dimensions)
    :return:
    """
    print('Loading word vectors')

    embed_index = dict()  # We create a dictionary of word -> embedding

    fname = 'glove.6B.{}d.txt'.format(dim)

    f = open(fname, encoding='utf8')  # Open file

    # In the dataset, each line represents a new word embedding
    # The line starts with the word and the embedding values follow
    for line in tqdm(f, desc='Loading Embeddings', unit='word'):
        values = line.split()
        # The first value is the word, the rest are the values of the embedding
        word = values[0]
        # Load embedding
        embedding = np.asarray(values[1:], dtype='float32')

        # Add embedding to our embedding dictionary
        embed_index[word] = embedding
    f.close()

    print('Found %s word vectors.' % len(embed_index))

    return embed_index

In [ ]:
embedding_dim = 200  

embeddings_index = load_glove_embeddings(dim=embedding_dim)

Loading Embeddings: 1633word [00:00, 16322.78word/s]

Loading word vectors


Loading Embeddings: 400000word [00:22, 17651.52word/s]

Found 400000 word vectors.


In [ ]:
def create_embeddings_matrix(emb_index: dict,
                             tokenizer: Tokenizer,
                             emb_dim: int = 200) -> np.ndarray:
    """

    :param emb_index: Embeddings Index
    :param tokenizer: Keras fitted tokenizer.
    :param emb_dim: Embeddings dimension.
    :return: A matrix of shape (nb_words, emb_dim) containing the globe embeddings.
    """
    assert emb_dim in [50, 100, 200, 300]

    # Create a matrix of all embeddings
    # (stacking=concatenating all the vectors)
    all_embs = np.stack(emb_index.values())  # .values() gets the all the arrays from the keys

    # Calculate mean
    emb_mean = all_embs.mean()
    # Calculate standard deviation
    emb_std = all_embs.std()

    print("Embeddings AVG: {} | STD: {}".format(emb_mean, emb_std))

    # We can now create an embedding matrix holding all word vectors.

    word_index = tokenizer.word_index

    # How many words are there actually. Because we may have requested X most common tokens
    # and the total tokens are X/2
    nb_words = min(max_words, len(word_index))

    # Create a random matrix with the same mean and std as the embeddings

    embedding_matrix = np.random.normal(emb_mean,  # mean
                                        emb_std,  # std
                                        (nb_words, emb_dim)) # shape of the matrix

    # The vectors need to be in the same position as their index.
    # Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on
    
    counter = 0
    
    # Loop over all words in the word index
    for word, i in word_index.items():  # .items() return a tuple with (word, word_index)

        # If we are above the amount of words we want to use we do nothing
        if i >= max_words:
            continue

        # Get the embedding vector for the word
        embedding_vector = emb_index.get(word)

        # If there is an embedding vector, put it in the embedding matrix
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            counter += 1
    
    print(f'Found {counter} pre-trained embeddings out of {nb_words}')
    
    return embedding_matrix

In [ ]:
embedding_matrix = create_embeddings_matrix(emb_index=embeddings_index,
                                            tokenizer=tokenizer,
                                            emb_dim=embedding_dim)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



Embeddings AVG: -0.008671863935887814 | STD: 0.38186216354370117
Found 33052 pre-trained embeddings out of 35000


In [ ]:
embedding_layer = Embedding(input_dim=max_words,
                            output_dim=embedding_dim,
                            input_length=max_len,
                            weights=[embedding_matrix],
                            trainable=False)

## In case of the 1st scenario, add class weights. Otherwise the class weights will be automatically equal to 1.

Since our data are now unbalanced, it is important to use class weights to increase the performance of models.

In [ ]:
# We insert the class weights according to our training data
y_integers = np.argmax(y_train_enc, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))

d_class_weights

{0: 1.0, 1: 1.0}

# 4. Model Training

## Dummy Classifier

In [ ]:
dummy_clf = DummyClassifier()
dummy_clf.fit(X_train_padded, label_encoded_y_train)


# make predictions for test data

y_pred = dummy_clf.predict(X_test)
predictions_dummy = [round(value) for value in y_pred]

# evaluate predictions

accuracy = accuracy_score(label_encoded_y_test, predictions_dummy)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 50.06%


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning:

The default value of strategy will change from stratified to prior in 0.24.



## 1. TF-IDF & XGBOOST Model

In [ ]:
from imblearn.pipeline import Pipeline
XGBoost = Pipeline([('vect', CountVectorizer(max_features=max_words, ngram_range = (1, 5))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', XGBClassifier(class_weight= d_class_weights)),
                    ])


parameters = {
    'clf__max_depth':(5,10)
}



/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [ ]:
start = time.time()
# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
xgboost = RandomizedSearchCV(XGBoost, param_distributions=parameters, cv=2, verbose=1, random_state=0, scoring=acc_scorer)
xgboost.fit(X_train, label_encoded_y_train)
xgboost_clf = xgboost.best_estimator_

# Fit the best algorithm to the data. 
xgboost_clf.fit(X_train, label_encoded_y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning:

The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  5.6min finished


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=35000, min_df=1,
                                 ngram_range=(1, 5), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=No...
                               class_weight={0: 1.0, 1: 1.0},
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=10,
                               min_child_weight=1, missing=N

In [ ]:
print("Total time: ", time.time() - start, "seconds")
xgboost_clf_time = time.time() - start

Total time:  739.7264566421509 seconds


In [ ]:
# make predictions for test data

y_pred = xgboost_clf.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions

accuracy = accuracy_score(label_encoded_y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 69.18%


## 2. TF-IDF & Linear SVM

In [ ]:
SVM = Pipeline([('vect', CountVectorizer(max_features=max_words, ngram_range=(1, 5))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC(random_state=0, dual = True, verbose=1,class_weight= d_class_weights)),
                    ])


parameters = {
    'clf__tol': (1e-3, 1e-4, 1e-5, 1e-6),
    'clf__max_iter': (1000, 10000),
    'clf__C': (0.1,1, 10)
}


In [ ]:
start = time.time()

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
SVM = RandomizedSearchCV(SVM, parameters, cv=2, verbose=1, random_state=0, scoring=acc_scorer)

SVM.fit(X_train, label_encoded_y_train)

# Set the clf to the best combination of parameters
svm_clf = SVM.best_estimator_

# Fit the best algorithm to the data. 
svm_clf.fit(X_train, label_encoded_y_train)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 11.7min finished


[LibLinear][LibLinear]

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=35000, min_df=1,
                                 ngram_range=(1, 5), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=0.1, class_weight={0: 1.0, 1: 1.0}, dual=True,
                           fit_intercept=True, intercept_scaling=1,
        

In [ ]:
print("Total time: ", time.time() - start, "seconds")
svm_clf_time = time.time() - start

Total time:  819.6908533573151 seconds


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
# = svm.LinearSVC(random_state=0, dual = True, tol=1e-5, verbose=1, max_iter=10000)
svm_clf.fit(X_train, label_encoded_y_train)# predict the labels on validation dataset
#SVM.fit(X_train_padded, label_encoded_y_train)# predict the labels on validation dataset
predictions_SVM = svm_clf.predict(X_test)# Use accuracy_score function to get the accuracy

#predictions_SVM = SVM.predict(X_test_padded)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, label_encoded_y_test)*100)

[LibLinear]SVM Accuracy Score ->  69.55502739816465


## 4. Multichannel Convolutional Neural Network (Keras Tokenizer + padding + Word2Vec embeddings)

In [ ]:
# Hyperparameters 

nb_classes = len(y_enc.categories_[0])

batch_size = 32

nb_epoch = 35

dropout_rate = 0.5

maxlen = max_len

nof_filters = 32

kernel_size = [3,4,5]

lr = 0.00001

In [ ]:
callbacks = [
    ModelCheckpoint('multi_cnn_model.h5', monitor='val_loss', save_best_only=True, mode='min')
]

In [ ]:
# define the model

def define_multichannel_cnn_model(length = max_len, vocab_size = max_words, nb_classes = nb_classes, dropout_rate = dropout_rate, nof_filters = nof_filters, kernel_size = kernel_size, lr = lr, embedding_dim = embedding_dim, embedding_matrix = embedding_matrix):
	inputs = Input(shape=(length,))
	embedding = Embedding(vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],trainable=False)(inputs)
  # channel 1
	conv1 = Conv1D(filters=nof_filters, kernel_size=kernel_size[0], activation='relu')(embedding)
	drop1 = Dropout(dropout_rate)(conv1)
	pool1 = MaxPooling1D(pool_size=3, padding='same')(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	conv2 = Conv1D(filters=nof_filters, kernel_size=kernel_size[1], activation='relu')(embedding)
	drop2 = Dropout(dropout_rate)(conv2)
	pool2 = MaxPooling1D(pool_size=3, padding='same')(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	conv3 = Conv1D(filters=nof_filters, kernel_size=kernel_size[2], activation='relu')(embedding)
	drop3 = Dropout(dropout_rate)(conv3)
	pool3 = MaxPooling1D(pool_size=2, padding='same')(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1,flat2,flat3])
	# flatten
	flat = Flatten()(merged)
	# add Dense Layers and Output Layer
	dense1 = Dense(250, activation='relu')(merged)
	dense2 = Dense(50, activation='relu')(dense1)
	dense3 = Dense(10, activation='relu')(dense2)
	outputs = Dense(2, activation='sigmoid')(dense3)
	model = Model(inputs=inputs, outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['acc', Recall(class_id=1, name='recall_1'), Precision(class_id=1, name='precision_1')])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel_cnn.png')
	return model

In [ ]:
multichannel_cnn_model = define_multichannel_cnn_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 48, 200)      7000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 46, 32)       19232       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 45, 32)       25632       embedding_1[0][0]                
______________________________________________________________________________________________

In [ ]:
start = time.time()

history_m_cnn = multichannel_cnn_model.fit(
    x=X_train_padded,           # features (as dense inputs)
    y=y_train_enc,           # labels
    batch_size=batch_size,   # define batch size
    epochs=nb_epoch,         # number of epochs
    verbose=1,               # the most extended verbosity level
    validation_data=(       
    X_val_padded,               # the validation split that we did before
    y_val_enc),
    callbacks=callbacks,
    class_weight = d_class_weights
)

Epoch 1/35
3030/3030 [==============================] - 112s 36ms/step - loss: 0.6864 - acc: 0.5513 - recall_1: 0.4738 - precision_1: 0.5504 - val_loss: 0.6672 - val_acc: 0.6031 - val_recall_1: 0.5050 - val_precision_1: 0.6353
Epoch 2/35
3030/3030 [==============================] - 108s 36ms/step - loss: 0.6650 - acc: 0.6039 - recall_1: 0.5602 - precision_1: 0.6125 - val_loss: 0.6512 - val_acc: 0.6261 - val_recall_1: 0.5093 - val_precision_1: 0.6635
Epoch 3/35
3030/3030 [==============================] - 107s 35ms/step - loss: 0.6489 - acc: 0.6301 - recall_1: 0.6002 - precision_1: 0.6377 - val_loss: 0.6427 - val_acc: 0.6365 - val_recall_1: 0.5485 - val_precision_1: 0.6644
Epoch 4/35
3030/3030 [==============================] - 108s 36ms/step - loss: 0.6419 - acc: 0.6351 - recall_1: 0.6089 - precision_1: 0.6392 - val_loss: 0.6367 - val_acc: 0.6459 - val_recall_1: 0.6178 - val_precision_1: 0.6546
Epoch 5/35
3030/3030 [==============================] - 110s 36ms/step - loss: 0.6351 - acc:

In [ ]:
print("Total time: ", time.time() - start, "seconds")
cnn_time = time.time() - start

Total time:  3736.9764325618744 seconds


In [ ]:
# plot training history
from matplotlib import pyplot
pyplot.plot(history_m_cnn.history['loss'], label='train')
pyplot.plot(history_m_cnn.history['val_loss'], label='test')
pyplot.legend()
pyplot.savefig("images/m_cnn_loss_per_epoch_plot.png", dpi=300)

<IPython.core.display.Javascript object>

In [ ]:
# evaluate and store on score variable on the TEST DATASET
# load the saved model
multi_cnn_model = load_model('multi_cnn_model.h5')


score = multi_cnn_model.evaluate(
    X_test_padded,    # features
    y_test_enc,              # labels
    batch_size=batch_size,   # batch size
    verbose=2                # the most extended verbose
)

947/947 - 8s - loss: 0.5962 - acc: 0.6900 - recall_1: 0.6376 - precision_1: 0.7103


## 5. Bi-Directional LSTM (Keras Tokenizer + padding + Word2Vec embeddings)

In [ ]:
# Hyperparameters 

batch_size = 64

nb_classes = len(y_enc.categories_[0])

nb_epoch = 15

dropout_rate = 0.5

maxlen = max_len

lr = 0.0001


callbacks = [
    ModelCheckpoint('rnn_model.h5', monitor='val_loss', save_best_only=True, mode='min')
]

In [ ]:
def define_lstm(maximum_words,max_seq_len, dropout_rate):

  rnn_model = Sequential()
  rnn_model.add(embedding_layer)


  rnn_model.add(Dropout(dropout_rate))

  # Add 2 bi-lstm layers

  rnn_model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)))

  rnn_model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)))

  rnn_model.add(Flatten())

  rnn_model.add(Dropout(dropout_rate))

  rnn_model.add(Dense(128, activation='relu'))

  rnn_model.add(Dense(nb_classes, activation='sigmoid'))


  print(rnn_model.summary())

  opt = keras.optimizers.Adam(learning_rate=lr)

  rnn_model.compile(optimizer=keras.optimizers.RMSprop(),
                      loss='binary_crossentropy',
                      metrics=['acc', Recall(class_id=1, name='recall_1'), Precision(class_id=1, name='precision_1')])
  
  plot_model(rnn_model, show_shapes=True, to_file='bi-lstm.png')

  return rnn_model

In [ ]:
lstm_model = define_lstm(max_words,maxlen, dropout_rate)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 48, 200)           7000000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 48, 200)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 48, 64)            59648     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 48, 64)            24832     
_________________________________________________________________
flatten_4 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 3072)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               3

In [ ]:
start = time.time()

history_lstm = lstm_model.fit(
    x=X_train_padded,           # features (as dense inputs)
    y=y_train_enc,           # labels
    batch_size=batch_size,   # define batch size
    epochs=nb_epoch,         # number of epochs
    verbose=1,               # the most extended verbosity level
    validation_data=(       
    X_val_padded,               # the validation split that we did before
    y_val_enc),
    callbacks=callbacks,
    class_weight = d_class_weights
)

Epoch 1/15
1515/1515 [==============================] - 578s 367ms/step - loss: 0.6586 - acc: 0.6119 - recall_1: 0.6006 - precision_1: 0.6149 - val_loss: 0.6103 - val_acc: 0.6715 - val_recall_1: 0.6962 - val_precision_1: 0.6632
Epoch 2/15
1515/1515 [==============================] - 536s 354ms/step - loss: 0.6234 - acc: 0.6589 - recall_1: 0.6483 - precision_1: 0.6617 - val_loss: 0.6009 - val_acc: 0.6821 - val_recall_1: 0.6353 - val_precision_1: 0.7013
Epoch 3/15
1515/1515 [==============================] - 536s 354ms/step - loss: 0.6135 - acc: 0.6696 - recall_1: 0.6479 - precision_1: 0.6778 - val_loss: 0.5903 - val_acc: 0.6933 - val_recall_1: 0.6941 - val_precision_1: 0.6925
Epoch 4/15
1515/1515 [==============================] - 540s 356ms/step - loss: 0.6055 - acc: 0.6784 - recall_1: 0.6421 - precision_1: 0.6928 - val_loss: 0.5856 - val_acc: 0.6974 - val_recall_1: 0.6908 - val_precision_1: 0.7002
Epoch 5/15
1515/1515 [==============================] - 541s 357ms/step - loss: 0.6027 -

In [ ]:
print("Total time: ", time.time() - start, "seconds")
bi_lstm_time = time.time() - start

Total time:  8286.917140722275 seconds


In [ ]:
# Refresh plot
pyplot.clf()
pyplot.cla()
pyplot.close()

pyplot.plot(history_lstm.history['loss'], label='train')
pyplot.plot(history_lstm.history['val_loss'], label='test')
pyplot.legend()
pyplot.savefig("images/rnn_loss_per_epoch_plot.png.png", dpi=300)

<IPython.core.display.Javascript object>

In [ ]:
# evaluate and store on score variable on the TEST DATASET
score = lstm_model.evaluate(
    X_test_padded,    # features
    y_test_enc,              # labels
    batch_size=batch_size,   # batch size
    verbose=2                # the most extended verbose
)

474/474 - 22s - loss: 0.5683 - acc: 0.7180 - recall_1: 0.6850 - precision_1: 0.7326


## 6. Att + LSTM (Keras Tokenizer + padding + Word2Vec embeddings)

In [ ]:
# Hyperparameters 

batch_size = 64

nb_classes = len(y_enc.categories_[0])

nb_epoch = 20

dropout_rate = 0.5

maxlen = max_len


lr = 0.0001

embedding_layer = Embedding(input_dim=max_words,
                            output_dim=embedding_dim,
                            input_length=max_len,
                            weights=[embedding_matrix],
                            trainable=False)

callbacks = [
    ModelCheckpoint('c-lstm_model.h5', monitor='val_loss', save_best_only=True, mode='min')
]

In [ ]:
!pip install keras-self-attention
from keras_self_attention import SeqSelfAttention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.49.0-cp37-none-any.whl size=19468 sha256=4bb75dcdd6f2b02bd39cca72859a8043803ef8a481bb8620b582444237071346
  Stored in directory: /root/.cache/pip/wheels/6f/9d/c5/26693a5092d9313daeae94db04818fc0a2b7a48ea381989f34
Successfully built keras-self-attention


In [ ]:
def define_att_lstm(length = max_len, vocab_size = max_words, nb_classes = nb_classes, dropout_rate = dropout_rate, nof_filters = nof_filters, kernel_size = kernel_size, lr = lr, embedding_dim = embedding_dim, embedding_matrix = embedding_matrix):
  embedding = Embedding(input_length=length, input_dim = vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],trainable=False, mask_zero=True)
  # Input with embeddings.
  att_lstm_model = Sequential()
  att_lstm_model.add(embedding)
  # Add Bi-lstm layer.
  att_lstm_model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate)))

  att_lstm_model.add(SeqSelfAttention(attention_activation='sigmoid'))

  att_lstm_model.add(Flatten())

  att_lstm_model.add(Dense(nb_classes, activation='sigmoid'))

  print(att_lstm_model.summary())

  opt = keras.optimizers.Adam(learning_rate=lr)

  att_lstm_model.compile(optimizer=opt,
                      loss='binary_crossentropy',
                      metrics=['acc', Recall(class_id=1, name='recall_1'), Precision(class_id=1, name='precision_1')])
  
  plot_model(att_lstm_model, show_shapes=True, to_file='cnn-lstm.png')

  return att_lstm_model

In [ ]:
att_lstm = define_att_lstm()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 48, 200)           7000000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 48, 64)            59648     
_________________________________________________________________
seq_self_attention (SeqSelfA (None, 48, 64)            4161      
_________________________________________________________________
flatten_5 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 6146      
Total params: 7,069,955
Trainable params: 69,955
Non-trainable params: 7,000,000
_________________________________________________________________
None


In [ ]:
start = time.time()

history_att_lstm = att_lstm.fit(
    x=X_train_padded,           # features (as dense inputs)
    y=y_train_enc,           # labels
    batch_size=batch_size,   # define batch size
    epochs=nb_epoch,         # number of epochs
    verbose=1,               # the most extended verbosity level
    validation_data=(       
    X_val_padded,               # the validation split that we did before
    y_val_enc),
    callbacks=callbacks,
    class_weight = d_class_weights
)

Epoch 1/20
1515/1515 [==============================] - 475s 305ms/step - loss: 0.6605 - acc: 0.6041 - recall_1: 0.6672 - precision_1: 0.5929 - val_loss: 0.6234 - val_acc: 0.6598 - val_recall_1: 0.6783 - val_precision_1: 0.6535
Epoch 2/20
1515/1515 [==============================] - 472s 312ms/step - loss: 0.6288 - acc: 0.6514 - recall_1: 0.6457 - precision_1: 0.6541 - val_loss: 0.6215 - val_acc: 0.6599 - val_recall_1: 0.5347 - val_precision_1: 0.7135
Epoch 3/20
1515/1515 [==============================] - 519s 342ms/step - loss: 0.6211 - acc: 0.6616 - recall_1: 0.6516 - precision_1: 0.6669 - val_loss: 0.6097 - val_acc: 0.6718 - val_recall_1: 0.6969 - val_precision_1: 0.6637
Epoch 4/20
1515/1515 [==============================] - 419s 276ms/step - loss: 0.6128 - acc: 0.6692 - recall_1: 0.6640 - precision_1: 0.6723 - val_loss: 0.6063 - val_acc: 0.6768 - val_recall_1: 0.6152 - val_precision_1: 0.7013
Epoch 5/20
1515/1515 [==============================] - 418s 276ms/step - loss: 0.6099 -

In [ ]:
print("Total time: ", time.time() - start, "seconds")
att_lstm_time = time.time() - start

Total time:  8610.695014238358 seconds


In [ ]:
# Refresh plot
pyplot.clf()
pyplot.cla()
pyplot.close()


pyplot.plot(history_att_lstm.history['loss'], label='train')
pyplot.plot(history_att_lstm.history['val_loss'], label='test')
pyplot.legend()
pyplot.savefig('images/att+lstm_loss_per_epoch_plot.png', dpi=300)

<IPython.core.display.Javascript object>

In [ ]:
# evaluate and store on score variable on the TEST DATASET
score = att_lstm.evaluate(
    X_test_padded,    # features
    y_test_enc,              # labels
    batch_size=batch_size,   # batch size
    verbose=2                # the most extended verbose
)

474/474 - 20s - loss: 0.5739 - acc: 0.7075 - recall_1: 0.7106 - precision_1: 0.7064


# 5. Results

## Classifier & Model Performance

In [ ]:
def create_cf_matrix(actual, predicted):
  cf_matrix = confusion_matrix(actual, predicted)

  group_names = ['True Neg','False Pos','False Neg','True Pos']
  group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]

  labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_names,group_counts)]

  labels = np.asarray(labels).reshape(2,2)
  plot_cf_matrix = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

  return plot_cf_matrix
                              



> XGboost Performance



In [ ]:
# Make predictions
predictions_xgboost = xgboost_clf.predict(X_test)
# Classification Report
print('Classification Report')
print(metrics.classification_report(label_encoded_y_test, predictions_xgboost))
# Confussion Matrix
print('Confussion Matrix')
xgboost = create_cf_matrix(label_encoded_y_test, predictions_xgboost)

Classification Report
              precision    recall  f1-score   support

           0       0.67      0.77      0.71     15147
           1       0.73      0.61      0.67     15147

    accuracy                           0.69     30294
   macro avg       0.70      0.69      0.69     30294
weighted avg       0.70      0.69      0.69     30294

Confussion Matrix




> SVM Classifier Performance



In [ ]:
# Make predictions
predictions_svm = svm_clf.predict(X_test)
# Classification Report
print('Classification Report')
print(metrics.classification_report(label_encoded_y_test, predictions_svm))
# Confussion Matrix
print('Confussion Matrix')
xgboost = create_cf_matrix(label_encoded_y_test, predictions_svm)

Classification Report
              precision    recall  f1-score   support

           0       0.68      0.74      0.71     15147
           1       0.71      0.65      0.68     15147

    accuracy                           0.70     30294
   macro avg       0.70      0.70      0.69     30294
weighted avg       0.70      0.70      0.69     30294

Confussion Matrix


Multichannel CNN Performance

In [ ]:
# Make predictions
predictions_m_cnn = np.argmax(multi_cnn_model.predict(X_test_padded), axis=-1)
# Classification Report
print(metrics.classification_report(label_encoded_y_test, predictions_m_cnn))
# Confussion Matrix
cnn_cf_matrix = create_cf_matrix(label_encoded_y_test, predictions_m_cnn)

              precision    recall  f1-score   support

           0       0.67      0.74      0.70     15147
           1       0.71      0.64      0.67     15147

    accuracy                           0.69     30294
   macro avg       0.69      0.69      0.69     30294
weighted avg       0.69      0.69      0.69     30294





> BI-LSTM Performance



In [ ]:
# Make predictions
predictions_lstm = np.argmax(lstm_model.predict(X_test_padded), axis=-1)
# Classification Report
print(metrics.classification_report(label_encoded_y_test, predictions_lstm))
# Confussion Matrix
lstm_cf_matrix = create_cf_matrix(label_encoded_y_test, predictions_lstm)

              precision    recall  f1-score   support

           0       0.70      0.75      0.73     15147
           1       0.73      0.68      0.71     15147

    accuracy                           0.72     30294
   macro avg       0.72      0.72      0.72     30294
weighted avg       0.72      0.72      0.72     30294





> Att + BI-LSTM Performance



In [ ]:
# Make predictions
predictions_att_lstm = np.argmax(att_lstm.predict(X_test_padded), axis=-1)
# Classification Report
print(metrics.classification_report(label_encoded_y_test, predictions_att_lstm))
# Confussion Matrix
att_lstm_cf_matrix = create_cf_matrix(label_encoded_y_test, predictions_att_lstm)

              precision    recall  f1-score   support

           0       0.71      0.71      0.71     15147
           1       0.71      0.71      0.71     15147

    accuracy                           0.71     30294
   macro avg       0.71      0.71      0.71     30294
weighted avg       0.71      0.71      0.71     30294



## Classifier & Model Comparison

In [ ]:
Names = ['Dummy Classifier', 'XGboost Model', 'Linear SVM Classifier', 'CNN-multichannel', 'Bi-LSTM', 'Att+Bi-LSTM']

accuracy_dummy = accuracy_score(label_encoded_y_test, predictions_dummy)
accuracy_xgboost = accuracy_score(label_encoded_y_test, predictions_xgboost)
accuracy_svm = accuracy_score(label_encoded_y_test, predictions_svm)
accuracy_cnn = accuracy_score(label_encoded_y_test, predictions_m_cnn)
accuracy_lstm = accuracy_score(label_encoded_y_test, predictions_lstm)
accuracy_att_lstm = accuracy_score(label_encoded_y_test, predictions_att_lstm)
Accuracy = [accuracy_dummy, accuracy_xgboost, accuracy_svm, accuracy_cnn, accuracy_lstm, accuracy_att_lstm]

Time = [1, xgboost_clf_time, svm_clf_time, cnn_time, bi_lstm_time ,att_lstm_time]


In [ ]:
f1_dummy = f1_score(label_encoded_y_test, predictions_dummy)
f1_xgboost = f1_score(label_encoded_y_test, predictions_xgboost)
f1_svm = f1_score(label_encoded_y_test, predictions_svm)
f1_cnn = f1_score(label_encoded_y_test, predictions_m_cnn)
f1_lstm = f1_score(label_encoded_y_test, predictions_lstm)
f1_att_lstm = f1_score(label_encoded_y_test, predictions_att_lstm)

f1_score_total = [f1_dummy, f1_xgboost, f1_svm, f1_cnn, f1_lstm, f1_att_lstm]

In [ ]:
color = list(range(20, 26))
color

[20, 21, 22, 23, 24, 25]

In [ ]:
temp_df_1 = pd.DataFrame({'Name of Classifier': Names, 'Accuracy': Accuracy, 'Color': Accuracy})

temp_df_2 = pd.DataFrame({'Name of Classifier': Names, 'Time (seconds) required to fit the data': Time, 'Color': Time})

temp_df_3 = pd.DataFrame({'Name of Classifier': Names, 'F1 Score': f1_score_total, 'Color': f1_score_total})

In [ ]:
fig_1 = go.Bar(x=temp_df_1['Name of Classifier'], y=temp_df_1['Accuracy'], marker_color=temp_df_1['Color'], width=[0.6, 0.6, 0.6, 0.6, 0.6, 0.6])
fig_2 = go.Bar(x=temp_df_2['Name of Classifier'], y=temp_df_2['Time (seconds) required to fit the data'], marker_color=temp_df_2['Color'], width=[0.6, 0.6, 0.6, 0.6, 0.6, 0.6])
fig_3 = go.Bar(x=temp_df_3['Name of Classifier'], y=temp_df_3['F1 Score'], marker_color=temp_df_3['Color'], width=[0.6, 0.6, 0.6, 0.6, 0.6, 0.6])

In [ ]:
# Subplots.
fig = make_subplots(rows=1, cols=3)
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=[
        'Test Accuracy (%)', 
        'Training & Parameter Tuning Time(seconds)', 
        'F1 Score (%)'
    ])

fig.add_trace(fig_1, row=1, col=1)
fig.add_trace(fig_2, row=1, col=2)
fig.add_trace(fig_3, row=1, col=3)

fig.update_layout(height=720, width=1280, template='plotly_white',showlegend=False)

plot(fig, filename="images/results_1.html", auto_open=False)

fig.show()

Roc Curve for neural network models

In [ ]:
from sklearn.metrics import roc_curve, auc

def create_roc_curve(test,pred):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(2):
        fpr[i], tpr[i], _ = roc_curve(test, pred)
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure()
    plt.plot(fpr[1], tpr[1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
    return plt

> Compare Results to the CoNLL-2010 Shared Task Results

In [ ]:
from random import randrange
!pip install colormap
!pip install easydev
from colormap import rgb2hex

  Created wheel for colormap: filename=colormap-1.0.3-cp37-none-any.whl size=15561 sha256=090abc4892c0a692b8ea37d551ff8712626c23c66f6055baefe43b5e29f64791
  Stored in directory: /root/.cache/pip/wheels/b5/c4/23/5f5d3e77fb0d00f44ee147a9cbb3acb641b9ebe2f3d5406c66
Successfully built colormap
     |████████████████████████████████| 51kB 2.6MB/s 
  Created wheel for easydev: filename=easydev-0.11.0-cp37-none-any.whl size=64244 sha256=db43a4ef80285181d0df6a731ae8c1ca53add9a5a3fb254c4c0e955b31a8127c
  Stored in directory: /root/.cache/pip/wheels/17/69/b5/771949d8100a30aa33e4bc5d0ff77eced9063284891ce26898
Successfully built easydev


In [ ]:
colors = []
for i in range(1,18):
    x = random.uniform(0, 1) 
    colors.append(x)
    
my_colors = []
for i in range(1,7):
    y = rgb2hex(randrange(256), randrange(256), randrange(256))
    my_colors.append(y)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

precision_total = [precision_score(label_encoded_y_test, predictions_dummy), 
                               precision_score(label_encoded_y_test, predictions_xgboost),
                               precision_score(label_encoded_y_test, predictions_svm),
                               precision_score(label_encoded_y_test, predictions_m_cnn),
                               precision_score(label_encoded_y_test, predictions_lstm),
                               precision_score(label_encoded_y_test, predictions_att_lstm)]


recall_total = [recall_score(label_encoded_y_test, predictions_dummy), 
                            recall_score(label_encoded_y_test, predictions_xgboost),
                            recall_score(label_encoded_y_test, predictions_svm),
                            recall_score(label_encoded_y_test, predictions_m_cnn),
                            recall_score(label_encoded_y_test, predictions_lstm),
                            recall_score(label_encoded_y_test, predictions_att_lstm)]


df_all_results = pd.DataFrame({'Name': Names, 'Precision': precision_total, 'Recall': recall_total, 'F1 Score': f1_score_total, 'Color': my_colors})


df_all_results['F1 Score'] = round(df_all_results['F1 Score']*100)
df_all_results['Precision'] = round(df_all_results['Precision']*100)
df_all_results['Recall'] = round(df_all_results['Recall']*100)

df_all_results = df_all_results.iloc[1:]

In [ ]:
# Add results from the participants of CoNLL-2010 Shared Task.
r1 = ['Georgescul', 72.0, 51.7,60.2, colors[0]]
r2 = ['Ji', 62.7, 55.3, 58.7, colors[1]]
r3 = ['Chen', 68.0, 49.7, 57.4, colors[2]]
r4 = ['Morante', 80.6, 44.5, 57.3, colors[3]]
r5 = ['Zhang', 76.6, 44.4, 56.2, colors[4]]
r6 = ['Zheng', 76.3, 43.6, 55.5, colors[5]]
r7 = ['Täckström', 78.3, 42.8, 55.4, colors[6]]
r8 = ['Mamani Sánchez', 68.3, 46.2, 55.1, colors[7]]
r9 = ['Tang', 82.3, 41.4, 55.0, colors[8]]
r10 = ['Kilicoglu', 67.9, 46.0, 54.9, colors[9]]
r11 = ['Tjong Kim Sang', 74.0, 43.0, 54.4, colors[10]]
r12 = ['Clausen', 75.1, 42.0, 53.9, colors[11]]
r13 = ['Özgür', 59.4, 47.9, 53.1, colors[12]]
r14 = ['Zhou', 85.3, 36.5, 51.1, colors[13]]
r15 = ['Li', 88.4, 31.9, 46.9, colors[14]]
r16 = ['Prabhakaran', 88.0, 28.4, 43.0, colors[15]]
r17 = ['Ji', 94.2, 6.6, 12.3, colors[16]]

In [ ]:
df_all_results = df_all_results.append(pd.Series(r1, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r2, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r3, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r4, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r5, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r6, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r7, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r8, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r9, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r10, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r11, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r12, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r13, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r14, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r15, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r16, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)
df_all_results = df_all_results.append(pd.Series(r17, index=['Name','Precision','Recall','F1 Score','Color']), ignore_index=True)

In [ ]:
df_all_results = df_all_results.set_index('Name')

In [ ]:
c = df_all_results['Color'].astype(str)

del df_all_results['Color']

In [ ]:
fig = plt.figure(figsize=(24,14))
df_all_results.T.plot(kind='bar', ax = fig.gca(), color = c)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize ='large')
plt.title('Model Comparisons with CoNLL-2010 Shared Task\'s Results', size = 25)
plt.xticks(size = 20)
plt.yticks(size = 15)
fig.savefig('images/results_2.png')
fig.show()

<IPython.core.display.Javascript object>